In [1]:
import pandas as pd
import numpy as np

train = []
labels = []

dfPredict = pd.read_csv("train.csv", header = 0, low_memory=False)
dfBackground = pd.read_csv("onlyCs.csv", header = 0, low_memory=False)
# GPA
rows = dfPredict.shape[0]
for i in range(rows):

    score = unicode("gpa", "utf-8")
    key = unicode("challengeID", "utf-8")
    gpa = dfPredict.loc[dfPredict.index[[i]], score]
    labels.append(gpa.values)
    
    # Gets corresponding data for each challengeID in training set
    challengeID = dfPredict.loc[dfPredict.index[[i]], key].values
    data = dfBackground.loc[dfBackground.index[[i - 1]]]
    
    # Removes from the array the id
    withoutID = data.drop(["challengeID"], axis=1)
    
    train.append(withoutID.values)


In [2]:
import math
newTrain = []
newLabels = []

for i in range(len(labels)):
    if not math.isnan(labels[i][0]):
        newTrain.append(train[i][0])
        newLabels.append(labels[i][0])


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
transformTrain = []
tobin = []
for arr in newTrain:
    for each in arr:
        if type(each) is str:
            tobin.append(each)
#             y = le.fit_transform(each)
#             transformTrain.append(y)

trans = le.fit_transform(tobin)

count = 0
for arr in newTrain:

    for each in arr:
        if type(each) is str:
            y = trans[count]
            count = count + 1
            transformTrain.append(y)
        else:
            transformTrain.append(each)
transformTrain = np.reshape(np.array(transformTrain), (len(newTrain),-1))

In [4]:
gpas = {1.0: 1, 1.25: 2, 1.50: 3, 1.75: 4, 2.0: 5, 2.25: 6, 2.50:7, 2.75:8, 3.0:9, 3.25:10, 3.50:11, 3.75: 12, 4.0:13}

In [5]:
classLabels = []
for each in newLabels:
    classLabels.append(gpas[each])

In [15]:
from sklearn.linear_model import RidgeClassifier
lr = RidgeClassifier()
lr.fit(transformTrain, classLabels)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [16]:
lr.score(transformTrain, classLabels)

0.81545064377682408

In [17]:
# prediction for all samples

rows = dfBackground.shape[0]
allSamples = []
for i in range(rows):
    
    data = dfBackground.loc[dfBackground.index[[i - 1]]]
    allSamples.append(data.drop(["challengeID"], axis=1).values[0])
    


In [18]:
transformSamples = []


tobin = []
for arr in allSamples:
    for each in arr:
        if type(each) is str:
            tobin.append(each)
#             y = le.fit_transform(each)
#             transformTrain.append(y)

trans = le.fit_transform(tobin)

count = 0
for arr in allSamples:

    for each in arr:
        if type(each) is str:
            y = trans[count]
            count = count + 1
            transformSamples.append(y)
        else:
            transformSamples.append(each)
transformSamples = np.reshape(np.array(transformSamples), (len(allSamples),-1))

In [19]:
out = lr.predict(transformSamples)
inv_gpas = {v: k for k, v in gpas.iteritems()}
transOut = []
for each in out:
    transOut.append(inv_gpas[each])


In [20]:
dfResults = pd.read_csv("prediction.csv", header = 0, low_memory=False)

In [21]:
dfResults["gpa"] = np.array(transOut)

In [22]:
dfResults.to_csv('prediction.csv', index=False)